In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import torch
print("pytorch ver . ",torch.__version__)
print("Is CUDA available?",torch.cuda.is_available())
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
import torch.utils.data as Data
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import xarray as xr
import os
os.environ['R_HOME'] = '/home/luhung3080/miniconda3/envs/chou/lib/R'
from rpy2.robjects import r, numpy2ri
numpy2ri.activate()
from rpy2.robjects.packages import importr
sinkr = importr('sinkr')
import Nio
print ("pynio ver . ",Nio.__version__)

/home/luhung3080/miniconda3/envs/chou/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


pytorch ver .  1.11.0+cu113
Is CUDA available? True
pynio ver .  1.5.5


In [2]:
data=pd.read_csv('/home/luhung3080/Desktop/PycharmProjects/Data/newdata/result.csv')

In [3]:
###
#cal_PMf
###
u=np.zeros([605,5183])
for i in range (0,605):
    a=np.array(data['VALUE'][5183*i:5183*i+5183])
    for j in range(0,5183):
        if a[j]=='NaN' :
            a[j]=np.nan
    u[i]=a.T
print(np.shape(u))
###
#obs_PMf
###
v=np.zeros([605,5183])
for i in range (0,605):
    a=np.array(data['OBS_VALUE'][5183*i:5183*i+5183])
    for j in range(0,5183):
        if a[j]=='NaN' :
            a[j]=np.nan
        if a[j]=='A' :
            a[j]=np.nan
        if a[j]=='*' :
            a[j]=np.nan
        if a[j]=='x' :
            a[j]=np.nan  
    v[i]=a.T

(605, 5183)


In [4]:
XRestruct=sinkr.dineof(u)
YRestruct=sinkr.dineof(v)

[1] "1 EOF ; RMS = 7.78530847"
[1] "1 EOF ; RMS = 7.7814125"
[1] "1 EOF ; RMS = 7.78138028"
[1] "1 EOF ; RMS = 7.78137947"
[1] "2 EOF ; RMS = 6.86645158"
[1] "2 EOF ; RMS = 6.86623009"
[1] "2 EOF ; RMS = 6.86623234"
[1] "3 EOF ; RMS = 6.53132364"
[1] "3 EOF ; RMS = 6.53141588"
[1] "4 EOF ; RMS = 6.274337"
[1] "4 EOF ; RMS = 6.27479489"
[1] "5 EOF ; RMS = 6.09927704"
[1] "5 EOF ; RMS = 6.10044296"
[1] "6 EOF ; RMS = 5.93199438"
[1] "6 EOF ; RMS = 5.93256512"
[1] "7 EOF ; RMS = 5.8063273"
[1] "7 EOF ; RMS = 5.80707008"
[1] "8 EOF ; RMS = 5.67541817"
[1] "8 EOF ; RMS = 5.67550394"
[1] "9 EOF ; RMS = 5.56763645"
[1] "9 EOF ; RMS = 5.5678256"
[1] "10 EOF ; RMS = 5.4598999"
[1] "10 EOF ; RMS = 5.46032904"
[1] "11 EOF ; RMS = 5.3663833"
[1] "11 EOF ; RMS = 5.36642172"
[1] "12 EOF ; RMS = 5.27728264"
[1] "12 EOF ; RMS = 5.27751306"
[1] "13 EOF ; RMS = 5.18525883"
[1] "13 EOF ; RMS = 5.18528898"
[1] "14 EOF ; RMS = 5.11321296"
[1] "14 EOF ; RMS = 5.11349983"
[1] "15 EOF ; RMS = 5.05407122"
[1] 

[1] "6 EOF ; RMS = 6.22387256"
[1] "6 EOF ; RMS = 6.22386812"
[1] "7 EOF ; RMS = 6.09070755"
[1] "7 EOF ; RMS = 6.09201364"
[1] "8 EOF ; RMS = 5.96107269"
[1] "8 EOF ; RMS = 5.96074222"
[1] "8 EOF ; RMS = 5.96075063"
[1] "9 EOF ; RMS = 5.87693636"
[1] "9 EOF ; RMS = 5.87731338"
[1] "10 EOF ; RMS = 5.78667064"
[1] "10 EOF ; RMS = 5.7866739"
[1] "11 EOF ; RMS = 5.71284227"
[1] "11 EOF ; RMS = 5.71377016"
[1] "12 EOF ; RMS = 5.6199033"
[1] "12 EOF ; RMS = 5.61921135"
[1] "12 EOF ; RMS = 5.61921261"
[1] "13 EOF ; RMS = 5.56931992"
[1] "13 EOF ; RMS = 5.57190791"
[1] "14 EOF ; RMS = 5.50356569"
[1] "14 EOF ; RMS = 5.50380709"
[1] "15 EOF ; RMS = 5.46322901"
[1] "15 EOF ; RMS = 5.46508733"
[1] "16 EOF ; RMS = 5.40478247"
[1] "16 EOF ; RMS = 5.40465555"
[1] "16 EOF ; RMS = 5.40468084"
[1] "17 EOF ; RMS = 5.35873426"
[1] "17 EOF ; RMS = 5.36014708"
[1] "18 EOF ; RMS = 5.32480972"
[1] "18 EOF ; RMS = 5.32492369"
[1] "19 EOF ; RMS = 5.29312947"
[1] "19 EOF ; RMS = 5.29543543"
[1] "20 EOF ; RMS =

In [5]:
XRestruct_Fun=np.array(XRestruct[0])
YRestruct_Fun=np.array(YRestruct[0])
print(np.shape(XRestruct_Fun))
print(np.shape(YRestruct_Fun))

(605, 5183)
(605, 5183)


In [8]:
a = np.zeros(605*5183)
b = np.zeros(605*5183)
for i in range (0,605):
    for j in range(0,5183):
        a[i*5183+j] = XRestruct_Fun[i][j]
        b[i*5183+j] = YRestruct_Fun[i][j]
print(np.shape(a))
print(np.shape(b))

(3135715,)
(3135715,)


In [11]:
# SITE_ID = data['SITE_ID']
# Date = data['IH']
# FCST_TIME = data['FCST_TIME']
# pm25_cal = data['VALUE']
# pm25_obs = data['OBS_VALUE']

In [7]:
newdata = data[['SITE_ID','IH','FCST_TIME','TAU']]
newdata.columns = ['SITE_ID','date','FCST_TIME','TAU']
newdata

,SITE_ID,date,FCST_TIME,TAU
0,EPA001,2020-02-24 08:00:00,2020-02-24 08:00:00,0
1,EPA001,2020-02-24 08:00:00,2020-02-24 09:00:00,1
2,EPA001,2020-02-24 08:00:00,2020-02-24 10:00:00,2
3,EPA001,2020-02-24 08:00:00,2020-02-24 11:00:00,3
4,EPA001,2020-02-24 08:00:00,2020-02-24 12:00:00,4
...,...,...,...,...
3135710,EPA080,2021-10-30 08:00:00,2021-11-02 04:00:00,68
3135711,EPA080,2021-10-30 08:00:00,2021-11-02 05:00:00,69
3135712,EPA080,2021-10-30 08:00:00,2021-11-02 06:00:00,70
3135713,EPA080,2021-10-30 08:00:00,2021-11-02 07:00:00,71


In [9]:
newdata.insert(4,'pm25_cal',a)
newdata.insert(5,'pm25_obs',b)
newdata

,SITE_ID,date,FCST_TIME,TAU,pm25_cal,pm25_obs
0,EPA001,2020-02-24 08:00:00,2020-02-24 08:00:00,0,7.2521,14.0
1,EPA001,2020-02-24 08:00:00,2020-02-24 09:00:00,1,4.9510,10.0
2,EPA001,2020-02-24 08:00:00,2020-02-24 10:00:00,2,4.4674,13.0
3,EPA001,2020-02-24 08:00:00,2020-02-24 11:00:00,3,4.6159,11.0
4,EPA001,2020-02-24 08:00:00,2020-02-24 12:00:00,4,3.9937,11.0
...,...,...,...,...,...,...
3135710,EPA080,2021-10-30 08:00:00,2021-11-02 04:00:00,68,3.6190,4.0
3135711,EPA080,2021-10-30 08:00:00,2021-11-02 05:00:00,69,3.7908,7.0
3135712,EPA080,2021-10-30 08:00:00,2021-11-02 06:00:00,70,4.0454,7.0
3135713,EPA080,2021-10-30 08:00:00,2021-11-02 07:00:00,71,3.9015,4.0


In [10]:
dd = newdata.drop(newdata[newdata['TAU'] == 0].index)
dd

,SITE_ID,date,FCST_TIME,TAU,pm25_cal,pm25_obs
1,EPA001,2020-02-24 08:00:00,2020-02-24 09:00:00,1,4.9510,10.0
2,EPA001,2020-02-24 08:00:00,2020-02-24 10:00:00,2,4.4674,13.0
3,EPA001,2020-02-24 08:00:00,2020-02-24 11:00:00,3,4.6159,11.0
4,EPA001,2020-02-24 08:00:00,2020-02-24 12:00:00,4,3.9937,11.0
5,EPA001,2020-02-24 08:00:00,2020-02-24 13:00:00,5,3.9602,9.0
...,...,...,...,...,...,...
3135710,EPA080,2021-10-30 08:00:00,2021-11-02 04:00:00,68,3.6190,4.0
3135711,EPA080,2021-10-30 08:00:00,2021-11-02 05:00:00,69,3.7908,7.0
3135712,EPA080,2021-10-30 08:00:00,2021-11-02 06:00:00,70,4.0454,7.0
3135713,EPA080,2021-10-30 08:00:00,2021-11-02 07:00:00,71,3.9015,4.0


In [11]:
dd.to_csv('/home/luhung3080/Desktop/PycharmProjects/Data/newdata/data_final.csv', index=False)

In [ ]:
history

In [7]:
dd = pd.read_csv('/home/luhung3080/Desktop/PycharmProjects/NCHUproject/Transformer/DLinear-main/dataset/ETTh1.csv')

In [8]:
dd

,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
0,2016-07-01 00:00:00,5.827,2.009,1.599,0.462,4.203,1.340,30.531000
1,2016-07-01 01:00:00,5.693,2.076,1.492,0.426,4.142,1.371,27.787001
2,2016-07-01 02:00:00,5.157,1.741,1.279,0.355,3.777,1.218,27.787001
3,2016-07-01 03:00:00,5.090,1.942,1.279,0.391,3.807,1.279,25.044001
4,2016-07-01 04:00:00,5.358,1.942,1.492,0.462,3.868,1.279,21.948000
...,...,...,...,...,...,...,...,...
17415,2018-06-26 15:00:00,-1.674,3.550,-5.615,2.132,3.472,1.523,10.904000
17416,2018-06-26 16:00:00,-5.492,4.287,-9.132,2.274,3.533,1.675,11.044000
17417,2018-06-26 17:00:00,2.813,3.818,-0.817,2.097,3.716,1.523,10.271000
17418,2018-06-26 18:00:00,9.243,3.818,5.472,2.097,3.655,1.432,9.778000


In [48]:
ss = pd.read_csv('/home/luhung3080/Desktop/PycharmProjects/NCHUproject/Transformer/DLinear-main/dataset/result_small.csv')

In [49]:
ss

,SITE_ID,date,FCST_TIME,pm25_cal,pm25_obs
0,EPA001,2020-02-24 08:00:00,2020-02-24 08:00:00,7.2521,14.0
1,EPA001,2020-02-24 08:00:00,2020-02-24 09:00:00,4.9510,10.0
2,EPA001,2020-02-24 08:00:00,2020-02-24 10:00:00,4.4674,13.0
3,EPA001,2020-02-24 08:00:00,2020-02-24 11:00:00,4.6159,11.0
4,EPA001,2020-02-24 08:00:00,2020-02-24 12:00:00,3.9937,11.0
...,...,...,...,...,...
3135710,EPA080,2021-10-30 08:00:00,2021-11-02 04:00:00,3.6190,4.0
3135711,EPA080,2021-10-30 08:00:00,2021-11-02 05:00:00,3.7908,7.0
3135712,EPA080,2021-10-30 08:00:00,2021-11-02 06:00:00,4.0454,7.0
3135713,EPA080,2021-10-30 08:00:00,2021-11-02 07:00:00,3.9015,4.0


In [50]:
new = ss[['date','pm25_cal','pm25_obs']]
new.columns = ['date','HUFL','OT']
new

,date,HUFL,OT
0,2020-02-24 08:00:00,7.2521,14.0
1,2020-02-24 08:00:00,4.9510,10.0
2,2020-02-24 08:00:00,4.4674,13.0
3,2020-02-24 08:00:00,4.6159,11.0
4,2020-02-24 08:00:00,3.9937,11.0
...,...,...,...
3135710,2021-10-30 08:00:00,3.6190,4.0
3135711,2021-10-30 08:00:00,3.7908,7.0
3135712,2021-10-30 08:00:00,4.0454,7.0
3135713,2021-10-30 08:00:00,3.9015,4.0


In [51]:
u=np.zeros([605*5183])
u

array([0., 0., 0., ..., 0., 0., 0.])

In [52]:
new.insert(2,'HULL',u)
new.insert(3,'MUFL',u)
new.insert(4,'MULL',u)
new.insert(5,'LUFL',u)
new.insert(6,'LULL',u)
new

,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
0,2020-02-24 08:00:00,7.2521,0.0,0.0,0.0,0.0,0.0,14.0
1,2020-02-24 08:00:00,4.9510,0.0,0.0,0.0,0.0,0.0,10.0
2,2020-02-24 08:00:00,4.4674,0.0,0.0,0.0,0.0,0.0,13.0
3,2020-02-24 08:00:00,4.6159,0.0,0.0,0.0,0.0,0.0,11.0
4,2020-02-24 08:00:00,3.9937,0.0,0.0,0.0,0.0,0.0,11.0
...,...,...,...,...,...,...,...,...
3135710,2021-10-30 08:00:00,3.6190,0.0,0.0,0.0,0.0,0.0,4.0
3135711,2021-10-30 08:00:00,3.7908,0.0,0.0,0.0,0.0,0.0,7.0
3135712,2021-10-30 08:00:00,4.0454,0.0,0.0,0.0,0.0,0.0,7.0
3135713,2021-10-30 08:00:00,3.9015,0.0,0.0,0.0,0.0,0.0,4.0


In [54]:
new

,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
0,2020-02-24 08:00:00,7.2521,0.0,0.0,0.0,0.0,0.0,14.0
1,2020-02-24 08:00:00,4.9510,0.0,0.0,0.0,0.0,0.0,10.0
2,2020-02-24 08:00:00,4.4674,0.0,0.0,0.0,0.0,0.0,13.0
3,2020-02-24 08:00:00,4.6159,0.0,0.0,0.0,0.0,0.0,11.0
4,2020-02-24 08:00:00,3.9937,0.0,0.0,0.0,0.0,0.0,11.0
...,...,...,...,...,...,...,...,...
3135710,2021-10-30 08:00:00,3.6190,0.0,0.0,0.0,0.0,0.0,4.0
3135711,2021-10-30 08:00:00,3.7908,0.0,0.0,0.0,0.0,0.0,7.0
3135712,2021-10-30 08:00:00,4.0454,0.0,0.0,0.0,0.0,0.0,7.0
3135713,2021-10-30 08:00:00,3.9015,0.0,0.0,0.0,0.0,0.0,4.0


In [55]:
new.to_csv('/home/luhung3080/Desktop/PycharmProjects/NCHUproject/Transformer/DLinear-main/dataset/ETTh1.csv', index=False)

In [61]:
da = pd.read_csv('/home/luhung3080/Desktop/PycharmProjects/NCHUproject/Transformer/DLinear-main/dataset/ETTh1.csv')

In [62]:
da

,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
0,2020-02-24 08:00:00,7.2521,0.0,0.0,0.0,0.0,0.0,14.0
1,2020-02-24 08:00:00,4.9510,0.0,0.0,0.0,0.0,0.0,10.0
2,2020-02-24 08:00:00,4.4674,0.0,0.0,0.0,0.0,0.0,13.0
3,2020-02-24 08:00:00,4.6159,0.0,0.0,0.0,0.0,0.0,11.0
4,2020-02-24 08:00:00,3.9937,0.0,0.0,0.0,0.0,0.0,11.0
...,...,...,...,...,...,...,...,...
3135710,2021-10-30 08:00:00,3.6190,0.0,0.0,0.0,0.0,0.0,4.0
3135711,2021-10-30 08:00:00,3.7908,0.0,0.0,0.0,0.0,0.0,7.0
3135712,2021-10-30 08:00:00,4.0454,0.0,0.0,0.0,0.0,0.0,7.0
3135713,2021-10-30 08:00:00,3.9015,0.0,0.0,0.0,0.0,0.0,4.0


In [63]:
da = da.drop(columns=['HULL','MUFL','MULL','LUFL','LULL'])
da

,date,HUFL,OT
0,2020-02-24 08:00:00,7.2521,14.0
1,2020-02-24 08:00:00,4.9510,10.0
2,2020-02-24 08:00:00,4.4674,13.0
3,2020-02-24 08:00:00,4.6159,11.0
4,2020-02-24 08:00:00,3.9937,11.0
...,...,...,...
3135710,2021-10-30 08:00:00,3.6190,4.0
3135711,2021-10-30 08:00:00,3.7908,7.0
3135712,2021-10-30 08:00:00,4.0454,7.0
3135713,2021-10-30 08:00:00,3.9015,4.0


In [64]:
da.to_csv('/home/luhung3080/Desktop/PycharmProjects/NCHUproject/Transformer/DLinear-main/dataset/ETTh1_de.csv', index=False)